In [1]:
import torch
import pandas as pd
import numpy as np
import datasets
from load_models_and_data import load_vocabulary, load_embeddings, text_to_embeddings, calc_cosine_sim, calculate_similarities
from tqdm import tqdm
tqdm.pandas()
from TwoTowerNN import QryTower, DocTower, TripletEmbeddingDataset, run_hyperparameter_tuning
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import Dataset, DataLoader,  SubsetRandomSampler
from sklearn.model_selection import KFold, train_test_split
import os
import wandb
from dotenv import load_dotenv

/root/MLX_Week2/mlx_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


API key loaded successfully


wandb: Currently logged in as: nnamdi-odozi (nnamdi-odozi-ave-actuaries) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
from datasets import load_dataset

# Loading datasets from Hugging Face
ds_soft_neg = load_dataset("cocoritzy/week_2_triplet_dataset_soft_negatives")
ds_hard_neg = load_dataset("cocoritzy/week_2_triplet_dataset_hard_negatives")


In [ ]:
# Paths to your files
embeddings_path = "./downloaded_model/embeddings.pt"
vocab_path = "./downloaded_model/tkn_ids_to_words.csv"

# Load embeddings and vocabulary
print("Loading embeddings and vocabulary...")
embeddings = load_embeddings(embeddings_path)
word_to_idx = load_vocabulary(vocab_path)

print(f"Loaded embeddings with shape: {embeddings.shape}")
print(f"Loaded vocabulary with {len(word_to_idx)} tokens")

# Example usage (uncomment when ready to test)
sample_text = "This is a test sentence"
embeddings_result = text_to_embeddings(sample_text, word_to_idx, embeddings)
print(f"Embedded text shape: {embeddings_result.shape}")

# Testing - Set numpy print options
np.set_printoptions(precision=4, suppress=True, threshold=10)  # threshold limits number of elements shown
numpy_array = embeddings_result.detach().numpy()
print("Embedding array with custom formatting:")
print(numpy_array)


In [ ]:
ds_soft_neg

In [ ]:
df_soft_neg  = pd.DataFrame(ds_soft_neg['train'])
df_hard_neg  = pd.DataFrame(ds_hard_neg['train'])

In [ ]:
embedded_query = text_to_embeddings(df_soft_neg['query'][0], word_to_idx, embeddings)
embedded_positive = text_to_embeddings(df_soft_neg['positive_passage'][0], word_to_idx, embeddings)
embedded_negative = text_to_embeddings(df_soft_neg['negative_passage'][0], word_to_idx, embeddings)

embedded_query.shape

In [ ]:
a = embedded_query.mean(dim=0)
b = embedded_positive.mean(dim=0)
c = embedded_negative.mean(dim=0)
a.shape


In [ ]:
import torch.nn.functional as F

cosine_similarity = F.cosine_similarity(a, c, dim=0)
print(f"Cosine similarity between query and positive passage: {cosine_similarity.item()}")

In [ ]:

# # Process the dataframe using apply just for first five rows
# print("Calculating similarities... This may take a while depending on dataframe size.")
# similarities = df_soft_neg[0:5].progress_apply(
#     lambda row: calculate_similarities(row, word_to_idx, embeddings), 
#     axis=1
# )

# # Join the similarities to the dataframe
# df_soft_neg_ext = pd.concat([df_soft_neg[0:5], similarities], axis=1)

# # Show a sample of the results
# #print(df_soft_neg_ext[['query_pos_sim', 'query_neg_sim', 'pos_neg_sim']].head())
#print(df_soft_neg_ext.head())
#print(df_soft_neg_ext.columns)

In [ ]:

# Process the dataframe using apply
print("Calculating similarities... This may take a while depending on dataframe size.")
similarities = df_soft_neg.progress_apply(
    lambda row: calculate_similarities(row, word_to_idx, embeddings), 
    axis=1
)

# Join the similarities to the dataframe
df_soft_neg_ext = pd.concat([df_soft_neg, similarities], axis=1)
print(df_soft_neg_ext.head())
# Show a sample of the results
#print(df_soft_neg_ext[['query_pos_sim', 'query_neg_sim', 'pos_neg_sim']].head())

#print(df_soft_neg_ext[['query_pos_sim', 'query_neg_sim', 'pos_neg_sim']].mean())

# Calculate how often the positive passage is ranked higher than negative
#higher_count = (df_soft_neg_ext['query_pos_sim'] > df_soft_neg_ext['query_neg_sim']).sum()
#total = len(df_soft_neg_ext)
#print(f"\nPositive passage ranked higher than negative: {higher_count} out of {total} ({higher_count/total:.2%})")



In [ ]:
# Process the dataframe using apply
print("Calculating similarities... This may take a while depending on dataframe size.")
similarities = df_hard_neg.progress_apply(
    lambda row: calculate_similarities(row, word_to_idx, embeddings), 
    axis=1
)

# Join the similarities to the dataframe
df_hard_neg_ext = pd.concat([df_hard_neg, similarities], axis=1)
print(df_hard_neg_ext.head())
# Show a sample of the results
#print(df_hard_neg_ext[['query_pos_sim', 'query_neg_sim', 'pos_neg_sim']].head())

#print(df_hard_neg_ext[['query_pos_sim', 'query_neg_sim', 'pos_neg_sim']].mean())

# Calculate how often the positive passage is ranked higher than negative
#higher_count = (df_hard_neg_ext['query_pos_sim'] > df_hard_neg_ext['query_neg_sim']).sum()
#total = len(df_hard_neg_ext)
#print(f"\nPositive passage ranked higher than negative: {higher_count} out of {total} ({higher_count/total:.2%})")



In [ ]:
df_all_neg_ext = pd.concat([df_soft_neg_ext, df_hard_neg_ext])
df_all_neg_ext.head()

In [ ]:
# Save DataFrames to pickle format
df_soft_neg_ext.to_pickle("data/df_soft_neg_ext.pkl")
df_hard_neg_ext.to_pickle("data/df_hard_neg_ext.pkl")
df_all_neg_ext.to_pickle("data/df_all_neg_ext.pkl")

In [4]:
# Function to load a DataFrame from pickle if the file exists
def load_df_if_exists(file_path):
    if os.path.exists(file_path):
        return pd.read_pickle(file_path)
    else:
        print(f"File not found: {file_path}")
        return None

# Load DataFrames
df_soft_neg_ext = load_df_if_exists("data/df_soft_neg_ext.pkl")
df_hard_neg_ext = load_df_if_exists("data/df_hard_neg_ext.pkl")
df_all_neg_ext = load_df_if_exists("data/df_all_neg_ext.pkl")


In [ ]:
df_soft_neg_ext.head()

### Twin Tower Network

In [ ]:
# Create tower instances
#qryTower = QryTower()
#docTower = DocTower()


# Define hyperparameters
batch_size = 128
num_epochs = 1 # adjust num of epochs here
dataset_size = len(df_all_neg_ext)  # or len(df_hard_neg_ext) depending on the dataset you want to use
steps_per_epoch = dataset_size // batch_size
total_steps = steps_per_epoch * num_epochs
learning_rate = 1e-3
embedding_dim = 128 
margin = 0.2 

In [ ]:
# Create the dataset
dataset = TripletEmbeddingDataset(df_all_neg_ext)

In [ ]:
dataloader = DataLoader(
    dataset,
    batch_size=batch_size,
    shuffle=True,
    #num_workers=2,  # Adjust based on your machine's capabilities
    pin_memory=True  # Set to True if using GPU
)

In [ ]:
qry = torch.randn(batch_size, embedding_dim)  # Query embeddings
pos = torch.randn(batch_size, embedding_dim)  # Positive doc embeddings
neg = torch.randn(batch_size, embedding_dim)  # Negative doc embeddings

#qry = df1['q']


# Set up the AdamW optimizer
optimizer = torch.optim.AdamW([
    {'params': qryTower.parameters()},
    {'params': docTower.parameters()}
], lr=learning_rate)

# Add learning rate scheduler (ReduceLROnPlateau)
scheduler = ReduceLROnPlateau(
    optimizer,
    mode='min',       # Reduce LR when monitored value stops decreasing
    factor=0.5,       # Multiply LR by this factor when reducing
    patience=2,       # Number of epochs with no improvement after which LR will be reduced
    verbose=True      # Print message when LR is reduced
)



In [ ]:
# Training loop (simplified example)
for epoch in range(num_epochs):
    qryTower.train()
    docTower.train()
    
    
    total_loss = 0
    for batch in dataloader:
        # Get embeddings from batch
        query_emb = batch['query']
        pos_emb = batch['positive']
        neg_emb = batch['negative']
        
        # Forward pass through towers
        query_encoded = qryTower(query_emb)
        pos_encoded = docTower(pos_emb)
        neg_encoded = docTower(neg_emb)
        
        # Calculate similarities
        pos_sim = torch.nn.functional.cosine_similarity(query_encoded, pos_encoded)
        neg_sim = torch.nn.functional.cosine_similarity(query_encoded, neg_encoded)
        
        # Triplet loss
        margin = margin
        loss = torch.clamp(margin - pos_sim + neg_sim, min=0).mean()
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item() * len(query_emb)
    
    # Calculate average loss
    avg_loss = total_loss / len(dataset)
    
    # Update scheduler
    scheduler.step(avg_loss)
    
    # Print epoch results
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}, "
          f"LR: {optimizer.param_groups[0]['lr']:.6f}")

In [5]:
# Run the hyperparameter tuning with your dataframe
best_params, final_qry_tower, final_doc_tower = run_hyperparameter_tuning(
    df_all_neg_ext,
    output_dims=[128],
    batch_sizes=[512],
    epochs=7
)

# Print the best parameters found
print(f"Best output dimension: {best_params['output_dim']}")
print(f"Best batch size: {best_params['batch_size']}")
print(f"Best validation loss: {best_params['val_loss']:.4f}")

Data splits: Train=95642 | Validation=31881 | Test=31881


--------------------------------------------------
Training with output_dim=128, batch_size=512
--------------------------------------------------


/root/MLX_Week2/mlx_env/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
Epoch 1/7 (Val): 100%|██████████| 63/63 [00:03<00:00, 17.92it/s]


Epoch 1/7, Train Loss: 0.1256, Val Loss: 0.1063, LR: 0.001000
New best model saved with validation loss: 0.1063


Epoch 2/7 (Val): 100%|██████████| 63/63 [00:03<00:00, 17.63it/s]


Epoch 2/7, Train Loss: 0.1025, Val Loss: 0.1031, LR: 0.001000
New best model saved with validation loss: 0.1031


...

Epoch 6/7, Train Loss: 0.0843, Val Loss: 0.0990, LR: 0.001000


Epoch 7/7 (Val): 100%|██████████| 63/63 [00:03<00:00, 19.07it/s]


Epoch 7/7, Train Loss: 0.0812, Val Loss: 0.0985, LR: 0.001000
New best model saved with validation loss: 0.0985
Final model saved at: checkpoints/final_model_20250423-161451/final_model_20250423-161451.pt


batch_size,▁
epoch,▁▂▃▅▆▇█▁▂▃▅▆▇█
learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁
output_dim,▁
train_loss,█▄▃▃▂▂▁▇▄▃▃▂▁▁
val_loss,█▅▅▄▃▃▃▃█▄▃▂▁▂▁
batch_size,512
epoch,7
learning_rate,0.001
output_dim,128
train_loss,0.08115


Best output dimension: 128
Best batch size: 512
Best validation loss: 0.1004


In [ ]:
def evaluate_model(qryTower, docTower, dataloader, device):
    qryTower.eval()
    docTower.eval()

    total = 0
    correct = 0

    for batch in dataloader:
        # Get embeddings from batch
        query_emb = batch['query']
        pos_emb = batch['positive']
        neg_emb = batch['negative']
        
        # Forward pass through towers
        query_encoded = qryTower(query_emb)
        pos_encoded = docTower(pos_emb)
        neg_encoded = docTower(neg_emb)
        
        # Calculate similarities
        pos_sim = torch.nn.functional.cosine_similarity(query_encoded, pos_encoded)
        neg_sim = torch.nn.functional.cosine_similarity(query_encoded, neg_encoded)

        correct += (pos_sim > neg_sim).sum().item()
        total += batch['query'].size(0)

    acc = correct / total
    print(f"Eval Accuracy (query closer to pos than neg): {acc:.4f}")
    return acc



total_loss = 0
    

In [ ]:
#print(f"Epoch {epoch+1}, Avg Loss: {total_loss / len(dataloader):.4f}")
evaluate_model(final_qry_tower, final_doc_tower, dataloader, device)

In [ ]:
query_emb = text_to_embeddings("What is RBA", word_to_idx, embeddings)
pos_emb = text_to_embeddings("What is RBA", word_to_idx, embeddings)

# Ensure tensors have at least two dimensions before applying mean
if query_emb.dim() == 1:
	query_emb = query_emb.unsqueeze(0)
if pos_emb.dim() == 1:
	pos_emb = pos_emb.unsqueeze(0)

query_emb = query_emb.mean(dim=0)
pos_emb = pos_emb.mean(dim=0)

print(torch.nn.functional.cosine_similarity(query_emb, pos_emb, dim=0))



In [ ]:

final_qry_tower.eval()
final_doc_tower.eval()
query_encoded = final_qry_tower(query_emb)
pos_encoded = final_doc_tower(pos_emb)
torch.nn.functional.cosine_similarity(query_encoded, pos_encoded, dim=0)

In [ ]:
# Code to upload final model to wandb
import wandb
import os
import time
from dotenv import load_dotenv

# Load your API key from config.txt
def load_api_key_from_config(config_path="config.txt"):
    try:
        with open(config_path, "r") as f:
            first_line = f.readline().strip()
            if "=" in first_line:
                api_key = first_line.split("=")[1].strip()
            else:
                api_key = first_line
        return api_key
    except FileNotFoundError:
        print(f"Config file not found at {config_path}")
        return None

# Set up wandb - only set API key if wandb.run doesn't exist yet
if wandb.run is None:
    api_key = load_api_key_from_config()
    if api_key:
        os.environ["WANDB_API_KEY"] = api_key
        wandb.login()
        print("Successfully logged in to Weights & Biases")
    else:
        print("Failed to load API key")

# Check if there's an active run, only initialize if needed
if wandb.run is None:
    run = wandb.init(
        project="twin-tower-model",
        name="final-model-summary",  # Changed name to indicate this is a summary
        config={
            "output_dim": best_params["output_dim"],
            "batch_size": best_params["batch_size"],
            "architecture": "Twin Tower Network",
            "dataset": "MS MARCO"
        }
    )
else:
    run = wandb.run
    # Update the run with additional metadata if needed
    run.config.update({
        "architecture": "Twin Tower Network",
        "dataset": "MS MARCO"
    })

# Upload the model with a timestamp to avoid conflicts
timestamp2 = int(time.time())
artifact_name = f"twin-tower-final-model-{timestamp2}"

model_artifact = wandb.Artifact(
    name=artifact_name, 
    type="model",
    description="Twin Tower model trained on full training data with optimal hyperparameters"
)

final_model_path = f"checkpoints/final_model/final_model.pt"
model_artifact.add_file(final_model_path)
wandb.log_artifact(model_artifact)

print(f"Final model uploaded to Weights & Biases project: {run.project}")

# Only finish the run if we created it in this script
if run.name == "final-model-summary":
    wandb.finish()